In [6]:
import json
import pandas as pd


In [9]:

file_path = "all_data/Xschlechteri_Nov2024/ncbi_dataset/ncbi_dataset/data/data_report.jsonl"

data = []


with open(file_path, 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file, start=1):
        try:
            # Parse the JSON object from each line
            json_obj = json.loads(line.strip())
            data.append(json_obj)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line {line_number}: {e}")


In [14]:
print(json.dumps(data[2], indent=4))  # Pretty-print the first JSON object


{
    "annotations": [
        {
            "annotationName": "Annotation submitted by TAIR and Araport",
            "annotationReleaseDate": "2022-10-20",
            "assemblyAccession": "GCF_000001735.4",
            "assemblyName": "TAIR10.1",
            "genomicLocations": [
                {
                    "genomicAccessionVersion": "NC_003071.7",
                    "genomicRange": {
                        "begin": "11678305",
                        "end": "11679919",
                        "orientation": "minus"
                    },
                    "sequenceName": "2"
                }
            ]
        }
    ],
    "chromosomes": [
        "2"
    ],
    "commonName": "thale cress",
    "description": "FAM210B-like protein, putative (DUF1279)",
    "geneId": "817272",
    "geneOntology": {
        "cellularComponents": [
            {
                "evidenceCode": "HDA",
                "goId": "GO:0009507",
                "name": "chloroplast",
       